# Question 2. IWAL algorithm implementation (50 points)

The purpose of this question is to implement Importance Weighted Active Learning (IWAL) algorithm. For this question, you will not use modAL, but instead will implement IWAL routine from scratch using scikit-learn, NumPy and native Python. 

In this question, we will use a simple synthetic dataset for a binary classification problem. Each data point has only 2 features. The dataset is provided in 2 files -- “data_iwal.npy”, which contains features and “labels_iwal.npy”, which contains labels. 

For simplicity, you will implement bootstrapping rejection sampling subroutine with logistic regression and hinge loss.   

𝐂𝐨𝐦𝐩𝐥𝐞𝐭𝐞 𝐭𝐡𝐞 𝐜𝐨𝐝𝐞 𝐮𝐧𝐝𝐞𝐫 ###𝐓𝐎 𝐃𝐎 𝐢𝐧 𝐞𝐚𝐜𝐡 𝐜𝐞𝐥𝐥 𝐚𝐧𝐝 𝐩𝐫𝐨𝐝𝐮𝐜𝐞 𝐭𝐡𝐞 𝐫𝐞𝐪𝐮𝐢𝐫𝐞𝐝 𝐩𝐥𝐨𝐭𝐬.  Feel free to define any helper functions as you see fit. You may import and use any modules in scikit-learn and NumPy to help with your implementations.

## Imports

Here we import necessary modules. Feel free to add something else here if you need it!

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import hinge_loss, log_loss
from sklearn.linear_model import LogisticRegression

## Reading data

Here we read the data and split it into train and test datasets. Train will be used to train our classification model and test will be used to validate the performance, monitor overfitting and compare the results of the model trained with Active Learning with the ones of the model trained from scratch. We set aside 1/3 of the dataset for validation.

In [2]:
X = np.load("data/q2/data_iwal.npy")
y = np.load("data/q2/labels_iwal.npy")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X[0:2])
print(y[0:2])

(134, 2)
(134,)
(66, 2)
(66,)
[[2.59193175 1.14706863]
 [1.7756532  1.15670278]]
[1 1]


In [31]:
lr = LogisticRegression().fit(X_train,y_train)
lr.predict(X_test)

array([1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1])

In [32]:
lr.predict_proba(X_test)

array([[9.84752005e-03, 9.90152480e-01],
       [9.81714776e-01, 1.82852241e-02],
       [9.99893900e-01, 1.06099698e-04],
       [9.44992134e-03, 9.90550079e-01],
       [2.80970174e-04, 9.99719030e-01],
       [3.76861843e-03, 9.96231382e-01],
       [9.93092609e-01, 6.90739085e-03],
       [3.37688223e-04, 9.99662312e-01],
       [9.77301111e-04, 9.99022699e-01],
       [3.72250743e-03, 9.96277493e-01],
       [1.00372007e-01, 8.99627993e-01],
       [6.04001582e-03, 9.93959984e-01],
       [4.85157010e-03, 9.95148430e-01],
       [8.81976245e-01, 1.18023755e-01],
       [5.49343178e-03, 9.94506568e-01],
       [9.99427820e-01, 5.72179703e-04],
       [9.98702473e-01, 1.29752680e-03],
       [9.98035276e-01, 1.96472392e-03],
       [9.99290314e-01, 7.09685867e-04],
       [9.84535308e-01, 1.54646922e-02],
       [2.93267103e-02, 9.70673290e-01],
       [2.57042586e-03, 9.97429574e-01],
       [2.02527261e-02, 9.79747274e-01],
       [8.96509819e-01, 1.03490181e-01],
       [1.794215

## Part 2.1
Type your answers for the theoretical questions below.

1. What is the idea behind IWAL algorithm?

**Your answer goes here**

2. What are the assumptions made for the IWAL algorithm?

**Your answer goes here**

3. What are the pros and cons of IWAL algorithm?

**Your answer goes here**

## Part 2.2 Implement IWAL algorithm

In this part you will implement a function that performs a single query of Algorithm 1 IWAL (subroutine rejection-sampling) from the paper. Below is the function description that you can follow in your implementation.

### Implemented Equations

$h_t = \underset{h \in H}{argmin}\sum_{(x,y,c)\in S_t}c \cdot l(h(x),y)$

In [5]:
### TODO: Implement Algorithm 1 from the paper
# implemented in custom Python package iwal_functions

## Part 2.3 Implement bootstrapping rejection sampling subroutine
In this part you will implement bootstrapping rejection sampling subroutine from the paper, section 7.2

### Implemented Equations
$p_t = p_{min}+(1-p_{min})[\underset{y;h_i,h_j \in H}{max}L(h_i(x),y)-L(h_j(x),y)]$

where $p_{min}$ is a lower bound on the sampling probability

In [6]:
# implemented in custom Python package iwal_functions

In [30]:
X = [[2.59193175, 1.14706863], [1.7756532, 1.15670278]]
y = [1, 0]
lr = LogisticRegression().fit(X, y)
x = np.asarray([3,1])
x = x.reshape(1, -1)


lr.predict_proba(x)

array([[0.42354956, 0.57645044]])

In [39]:
np.asarray([3, 1]).reshape(1, -1).shape

(1, 2)

## Part 2.4 Organize all implemented parts into a single pipeline
You implemented all parts of the IWAL algorithm with bootstrap rejection sampling. Now organize it into a pipeline.

In [ ]:
#?? isn't initializing H creating n_h classifiers?
#?? initialize history means to simply set up the empty dictionary?
#?? need to fit models first? what data do I use?
#?? don't understand bootstrapped data here
#?? which loss function to use here? why is log_loss also imported?

# create an initial training set
n_initial = 10
X_training, y_training = X_train[:n_initial], y_train[:n_initial]


# Initialize history and H
## TODO: your code here


# Create n_h classifiers and train them on bootstrapped data (data is sampled with replacement)
## TODO: your code goes here
H = []
n_h = 10
for i in range(n_h):
    lr = LogisticRegression()
    lr.fit(X_training, y_training)
    H.append(lr)

    
# initialize additional arguments
history = {'X': [],'y': [],'c': [],'Q': []}
selected = []
labels = [0,1]
p_min = 0.1  

# format as required by bootstrap function
bootstrap_arguments = {
    'H': H,
    'loss': hinge_loss,
    'labels': labels,
    'p_min': p_min
}

In [21]:
%%time

# my custom package
from packages.iwal import iwal_functions


# Perform queries and record loss
losses = []
n_query = 50
for t in range(n_query):
    
    ### TODO: your code goes here
    # select sample
    x_t = [[1,1]]     
    y_t = 1
    
    
    # derive optimal hypothesis
    h_t = iwal_functions.iwal_query(x_t,
                                    y_t,
                                    selected,
                                    iwal_functions.bootstrap, 
                                    history, 
                                    H, 
                                    hinge_loss, 
                                    labels, 
                                    bootstrap_arguments)
    
    
    #?? why use predict_proba
    # calculate loss and store for later
    loss_t = log_loss(y_test, h_t.predict_proba(X_test)) 
    losses.append(loss_t)
    

CPU times: user 2.98 ms, sys: 3.27 ms, total: 6.25 ms
Wall time: 4.71 ms


In [ ]:
plt.plot(losses)

## Part 2.5 Compare results of Active Learning vs No Active Learning

In this part you need to create object of the same scikit learning class and train it on randomly selected subset of data points and compare results of 2 classifiers. Comment on your observations

In [ ]:
# Compare to no Active Learning setting

## TODO: your code goes here